# SimPy

## Install Dependencies

In [10]:
!pip install -U simpy

## Import Dependenies

In [11]:
import simpy
from simpy import Environment
import random
import statistics
import numpy as np

## Call Center Simulation

In [16]:
customers_handled = 0

class CallCenter:
    def __init__(self,
                 env: Environment,
                 n_employees: int = 2,
                 avg_support_time_in_min: int = 5,
                 customer_interval_in_min: int = 2,
                 sim_time_in_min: int = 120):
        self.env = env
        # Resources are things that can only be used one at a time.
        self.n_employees = n_employees
        self.employees = simpy.Resource(env, n_employees)
        self.avg_support_time_in_min = avg_support_time_in_min
        self.customer_interval_in_min = customer_interval_in_min
        self.sim_time_in_min = sim_time_in_min

    def support(self, customer: str):
        random_time = max(1, np.random.normal(self.avg_support_time_in_min, 4))
        yield self.env.timeout(delay=random_time)
        print(f'Support finished for {customer} at {self.env.now:.2f}')
        
def customer(env: Environment, name: str, call_center: CallCenter):
    global customers_handled
    print(f'Customer "{name}" enters waiting queue at {env.now:.2f}.')
    
    with call_center.employees.request() as request:
        yield request
        print(f'Customer "{name}" enters call at {env.now:.2f}.')

        yield env.process(call_center.support(customer=name))
        print(f'Customer "{name}" left call at {env.now:.2f}.')

        customers_handled += 1

def setup(env: Environment):
    call_center: CallCenter = CallCenter(env=env)

    for i in range(1, 6):
        env.process(customer(env=env, name=call_center.n_employees, call_center=call_center))

    while True:
        yield env.timeout(delay=random.randint(call_center.customer_interval_in_min-1, call_center.customer_interval_in_min+1))
        i += 1
        env.process(customer(env=env, name=call_center.n_employees, call_center=call_center))

In [17]:
print('Starting call center simulation.')
env: Environment = simpy.Environment()

env.process(setup(env=env))
env.run(until=120)

print(f'{customers_handled} customers handled.')

Starting call center simulation.
Customer "2" enters waiting queue at 0.00.
Customer "2" enters waiting queue at 0.00.
Customer "2" enters waiting queue at 0.00.
Customer "2" enters waiting queue at 0.00.
Customer "2" enters waiting queue at 0.00.
Customer "2" enters call at 0.00.
Customer "2" enters call at 0.00.
Customer "2" enters waiting queue at 1.00.
Customer "2" enters waiting queue at 2.00.
Customer "2" enters waiting queue at 3.00.
Support finished for 2 at 3.38
Customer "2" left call at 3.38.
Customer "2" enters call at 3.38.
Customer "2" enters waiting queue at 4.00.
Support finished for 2 at 6.94
Customer "2" left call at 6.94.
Customer "2" enters call at 6.94.
Customer "2" enters waiting queue at 7.00.
Customer "2" enters waiting queue at 9.00.
Support finished for 2 at 10.93
Customer "2" left call at 10.93.
Customer "2" enters call at 10.93.
Customer "2" enters waiting queue at 11.00.
Support finished for 2 at 11.19
Customer "2" left call at 11.19.
Customer "2" enters cal